In [73]:
# Dependencies
import pandas as pd
import requests
import json
import os
import csv
import prettyprint as pp
# from apiclient import errors

# Google developer API key
from api_keys import gkey

In [74]:
world_happiness = "world happiness data.csv"

world_happiness_df = pd.read_csv(world_happiness, encoding="UTF-8")

 # base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [75]:
health_location_types = location_types_df.loc[location_types_df["sub_type"]
                                               =="Health",:]
health_types_list = []
health_location_types.reset_index(inplace=True)
for i in range(len(health_location_types)):
    health_types_list.append(health_location_types.loc[i,"location_types"])

health_types_list

['dentist', 'doctor', 'gym', 'hospital', 'pharmacy', 'physiotherapist']

In [76]:
base_url = "https://maps.googleapis.com/maps/api/geocode/json"
coord=[]
cities=["Helsinki","Oslo","Ottawa","Washington D.C","Rome","Tokyo","City of Victoria","Beijing","Dhaka"",New Delhi","Gaborone","Kabul"]
for city in cities:
    params = {"address": city, "key":gkey}
    response = requests.get(base_url, params=params)
    places_data = response.json()   
    lat=places_data["results"][0]["geometry"]["location"]["lat"]
    long=places_data["results"][0]["geometry"]["location"]["lng"]
    coord.append(f"{lat},{long}")
coord

['60.16985569999999,24.9383791',
 '59.9138688,10.7522454',
 '45.4215296,-75.69719309999999',
 '38.9071923,-77.0368707',
 '41.9027835,12.4963655',
 '35.6761919,139.6503106',
 '44.8608086,-93.660106',
 '39.90419989999999,116.4073963',
 '28.6461322,77.1964488',
 '-24.6282079,25.9231471',
 '34.5553494,69.207486']

In [77]:
test_list=[]
test_sub_list=[]

for x in range(10):
    test_sub_list.append(x)
    test_list.append(test_sub_list)
test_list

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]

In [90]:
#Use for loop to calculate hospitals

target_type ="hospital"
count_hospitals=[]

for lat in coord:
    params = {"location":lat,"key": gkey,"keyword":target_type, "radius":750}
    places_data = requests.get(base_url, params=params).json()
    count_hospitals.append(len(places_data["results"]))
    
count_hospitals

[3, 2, 1, 2, 0, 0, 0, 3, 6, 1, 0]

In [91]:
#Use for loop to calculate dentists

target_type ="dentist"
count_dentists=[]

for lat in coord:
    params = {"location":lat,"key": gkey,"keyword":target_type, "radius":750}
    places_data = requests.get(base_url, params=params).json()
    count_dentists.append(len(places_data["results"]))

count_dentists


[20, 20, 20, 20, 19, 13, 3, 0, 8, 1, 3]

In [47]:
health_types_list

['dentist', 'doctor', 'gym', 'hospital', 'pharmacy', 'physiotherapist']

In [92]:
#Use for loop to calculate doctors
target_type ="doctor"
count_doctors=[]

for lat in coord:
    params = {"location":lat,"key": gkey,"keyword":target_type, "radius":750}
    places_data = requests.get(base_url, params=params).json()
    count_doctors.append(len(places_data["results"]))

count_doctors


[20, 20, 20, 20, 20, 7, 2, 10, 20, 6, 2]

In [93]:
#Use for loop to calculate doctors
target_type ="gym"
count_gyms=[]

for lat in coord:
    params = {"location":lat,"key": gkey,"keyword":target_type, "radius":750}
    places_data = requests.get(base_url, params=params).json()
    count_gyms.append(len(places_data["results"]))

count_gyms

[20, 20, 20, 20, 10, 11, 2, 13, 20, 2, 2]

In [94]:
#Use for loop to calculate doctors
target_type ="pharmacy"
count_pharmacies=[]

for lat in coord:
    params = {"location":lat,"key": gkey,"keyword":target_type, "radius":750}
    places_data = requests.get(base_url, params=params).json()
    count_pharmacies.append(len(places_data["results"]))

count_pharmacies

[19, 20, 18, 20, 20, 20, 0, 11, 20, 4, 3]

In [95]:
#Use for loop to calculate doctors
target_type ="physiotherapist"
count_physio=[]

for lat in coord:
    params = {"location":lat,"key": gkey,"keyword":target_type, "radius":750}
    places_data = requests.get(base_url, params=params).json()
    count_physio.append(len(places_data["results"]))

    
count_physio

[9, 7, 13, 18, 16, 0, 5, 0, 1, 2, 0]

In [96]:
health_dict = {"Cities":cities,"Hospitals":count_hospitals,"Doctors":count_doctors,"Dentists":count_dentists,
              "Gyms":count_gyms,"Pharmacies":count_pharmacies,"Physiotherapists":count_physio}

health_data_df = pd.DataFrame(health_dict)
health_data_df

,Cities,Hospitals,Doctors,Dentists,Gyms,Pharmacies,Physiotherapists
0,Helsinki,3,20,20,20,19,9
1,Oslo,2,20,20,20,20,7
2,Ottawa,1,20,20,20,18,13
3,Washington D.C,2,20,20,20,20,18
4,Rome,0,20,19,10,20,16
5,Tokyo,0,7,13,11,20,0
6,City of Victoria,0,2,3,2,0,5
7,Beijing,3,10,0,13,11,0
8,"Dhaka,New Delhi",6,20,8,20,20,1
9,Gaborone,1,6,1,2,4,2
